In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow opencv-python matplotlib


In [3]:
import zipfile
import os

# Unzip the dataset (adjust the path to your zip file)
zip_path = '/content/drive/MyDrive/ML/detect_number_of_male_female_dataset.zip'
unzip_folder = '/content/dataset/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_folder)

# Assuming all images are in the same folder, we manually label and categorize them
# Create directories for male and female categories
os.makedirs('/content/dataset/male', exist_ok=True)
os.makedirs('/content/dataset/female', exist_ok=True)


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a data generator for data augmentation and preprocessing
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

# Prepare the train and validation generators
train_generator = datagen.flow_from_directory(
    '/content/dataset/',
    target_size=(150, 150),  # Resize images
    batch_size=32,
    class_mode='binary',  # We have two classes (male/female)
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/content/dataset/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 14000 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Initialize the CNN model
model = Sequential()

# First convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2, 2))

# Second convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Third convolutional layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten the output for the fully connected layers
model.add(Flatten())

# Fully connected layer
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Dropout to prevent overfitting

# Output layer (binary classification: male or female)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,034,177 (72.61 MB)

 Trainable params: 19,034,177 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10  # You can change the number of epochs based on performance
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


437/437 ━━━━━━━━━━━━━━━━━━━━ 31s 53ms/step - accuracy: 0.7160 - loss: 0.5691 - val_accuracy: 0.9221 - val_loss: 0.2096
Epoch 2/10
  1/437 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step - accuracy: 0.9688 - loss: 0.1685

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


437/437 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9688 - loss: 0.1685 - val_accuracy: 0.8750 - val_loss: 0.3473
Epoch 3/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 29s 41ms/step - accuracy: 0.9177 - loss: 0.2162 - val_accuracy: 0.9323 - val_loss: 0.1831
Epoch 4/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9375 - loss: 0.1362 - val_accuracy: 1.0000 - val_loss: 0.0537
Epoch 5/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 19s 39ms/step - accuracy: 0.9276 - loss: 0.1892 - val_accuracy: 0.9405 - val_loss: 0.1582
Epoch 6/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9375 - loss: 0.1186 - val_accuracy: 0.8750 - val_loss: 0.1830
Epoch 7/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 18s 40ms/step - accuracy: 0.9438 - loss: 0.1517 - val_accuracy: 0.9407 - val_loss: 0.1572
Epoch 8/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 0s 50us/step - accuracy: 0.8750 - loss: 0.2492 - val_accuracy: 0.8750 - val_loss: 0.2422
Epoch 9/10
437/437 ━━━━━━━━━━━━━━━━━━━━ 17s 39ms/step - accuracy: 0.9442 - loss: 0.1416 - val_accuracy: 0.94

In [7]:
# Save the trained model
model.save('/content/gender_classification_model.h5')